# Libraries

In [1]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [2]:
# Get the current user's name
user = getuser()

# Construct the path using the user's name
function_path = f"C:/Users/{user}/Documents/GitHub/tiebreak_wc/code/wiki/functions_uefa.ipynb"

# Run the notebook
%run $function_path

In [3]:


# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\in\wc_goals_men.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,year,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,Group A,1986,12:00CST,Bulgaria,Italy,1–1,Estadio Azteca,Mexico City,96000,Erik Fredriksson,...,85,0,1,1,0,0,0,0,NaN,31 May 1986
1,Group A,1986,12:00CST,Bulgaria,Italy,1–1,Estadio Azteca,Mexico City,96000,Erik Fredriksson,...,44,0,1,1,0,0,0,0,NaN,31 May 1986
2,Group A,1986,12:00CST,Argentina,South Korea,3–1,Estadio Olímpico Universitario,Mexico City,60000,Victoriano Sánchez Arminio,...,6,0,3,1,0,0,0,0,NaN,2 June 1986
3,Group A,1986,12:00CST,Argentina,South Korea,3–1,Estadio Olímpico Universitario,Mexico City,60000,Victoriano Sánchez Arminio,...,46,0,3,1,0,0,0,0,NaN,2 June 1986
4,Group A,1986,12:00CST,Argentina,South Korea,3–1,Estadio Olímpico Universitario,Mexico City,60000,Victoriano Sánchez Arminio,...,18,0,3,1,0,0,0,0,NaN,2 June 1986


# Clean, transfrom, create variables

## time

In [4]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## date

In [5]:
# Step 1: Remove any commas before the year in 'long_date'
df['long_date'] = df['long_date'].str.replace(r',\s*(\d{4})', r' \1', regex=True)

# Step 2: Convert 'long_date' to 'short_date' where 'short_date' is missing
df['short_date'] = df['short_date'].fillna(pd.to_datetime(df['long_date'], errors='coerce').dt.strftime('%Y-%m-%d'))


# Extract relevant columns

In [6]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

# Remove duplicates
goals_df = goals_df.drop_duplicates()

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
0,1986,Group A,Bulgaria,Italy,Bulgaria,85,1986-05-31,12:00,1–1
1,1986,Group A,Bulgaria,Italy,Italy,44,1986-05-31,12:00,1–1
2,1986,Group A,Argentina,South Korea,Argentina,6,1986-06-02,12:00,3–1
3,1986,Group A,Argentina,South Korea,Argentina,46,1986-06-02,12:00,3–1
4,1986,Group A,Argentina,South Korea,Argentina,18,1986-06-02,12:00,3–1


# Recreate Leauge Table after first two matchdays

In [7]:

# Assuming goals_df is your original dataset
agg_goals_before_last_day, goals_last_day_sorted = process_goals_data(goals_df)


In [8]:
home_games, away_games = aggregate_home_away_points(agg_goals_before_last_day)

In [9]:
# Filter for year == 2022 and stage == 'Group E'
home2022e = home_games[(home_games['year'] == 2022) & (home_games['stage'] == 'Group E')]
home2022e

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home
171,2022,Group E,Germany,1,2,0,1
172,2022,Group E,Japan,0,1,0,1
173,2022,Group E,Spain,8,1,4,2


In [10]:
# Filter for year == 2022 and stage == 'Group E'
away2022e = away_games[(away_games['year'] == 2022) & (away_games['stage'] == 'Group E')]
away2022e

,year,stage,away_team,goals_scored,goals_conceded,points_away,match_count_away
167,2022,Group E,Costa Rica,1,7,3,2
168,2022,Group E,Germany,1,1,1,1
169,2022,Group E,Japan,2,1,3,1


## aggregate data after first two matches 

In [11]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = uefa_before_last(home_games, away_games, agg_goals_before_last_day)

In [12]:
# Filtering all_games_before_last for the year 2022 and stage 'Group E'
group2022e = all_games_before_last[
    (all_games_before_last['year'] == 2022) & 
    (all_games_before_last['stage'] == 'Group E')
]

display(group2022e)

,year,stage,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,tie_won,standing
234,2022,Group E,Spain,8.0,1.0,4.0,7.0,2.0,no need,0,1
235,2022,Group E,Japan,2.0,2.0,3.0,0.0,2.0,Japan,1,2
236,2022,Group E,Costa Rica,1.0,7.0,3.0,-6.0,2.0,Japan,0,3
237,2022,Group E,Germany,2.0,3.0,1.0,-1.0,2.0,no need,0,4


# Recreate league table after last match day

### uefa criteria 

In [13]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    result = uefa_final_euro(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[[ 'year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]



=== Initial Standings for Year 1986, Group A Before Last Match Goals ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_points  before_last_game_standing
  Argentina           4.0                 4.0                   2.0                    2.0                      3.0                          1
   Bulgaria           3.0                 2.0                   2.0                    0.0                      2.0                          2
      Italy           3.0                 2.0                   2.0                    0.0                      2.0                          3
South Korea           2.0                 2.0                   4.0                   -2.0                      1.0                          4


Analyzing goal: 4 minute, Player team: Argentina, Home: Argentina, Away: Bulgaria

=== Updated Standings After This Goal ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_di

In [14]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\men\uefa\wc\tb_wc_uefa_men.xlsx'
changes_df_euro.to_excel(file_path, index=False)


# group composition tracking

In [15]:
# Initialize an empty list to store the results for each pair
all_composition_changes = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    composition_changes_df = track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_composition_changes.append(composition_changes_df)

# Concatenate all the results into a single DataFrame
final_composition_changes_df = pd.concat(all_composition_changes, ignore_index=True)

# Display the final DataFrame
display(final_composition_changes_df)




=== Initial Standings for Group A, 1986 (with 0-0 points added) ===
       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_standing
  Argentina           4.0                 4.0                   2.0                    2.0                          1
   Bulgaria           3.0                 2.0                   2.0                    0.0                          2
      Italy           3.0                 2.0                   2.0                    0.0                          3
South Korea           2.0                 2.0                   4.0                   -2.0                          4



=== Standings after goal at minute 4 in Group A, edition 1986 ===
       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_standing  tied_won
  Argentina           6.0                 5.0                   2.0                    3.0                   1         0
      Italy           3.0     

,year,stage,change_num,goal_time,home_team,away_team,scorer_team,new_top_teams,1st,2nd,3rd,scorer_nationality
0,1986,Group A,0,initial,None,None,None,"[Argentina, Italy, Bulgaria]",Argentina,Bulgaria,Italy,NaN
1,1986,Group B,0,initial,None,None,None,"[Paraguay, Belgium, Mexico]",Mexico,Paraguay,Belgium,NaN
2,1986,Group C,0,initial,None,None,None,"[Hungary, France, Soviet Union]",Soviet Union,France,Hungary,NaN
3,1986,Group D,0,initial,None,None,NaN,"[Brazil, Spain, Northern Ireland]",Brazil,Spain,Northern Ireland,NaN
4,1986,Group D,1,15,Northern Ireland,Brazil,NaN,"[Brazil, Spain, Algeria]",Brazil,Spain,Algeria,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...
109,2022,Group G,1,35,Serbia,Switzerland,NaN,"[Serbia, Brazil]",Brazil,Serbia,Switzerland,Serbia
110,2022,Group G,2,44,Serbia,Switzerland,NaN,"[Brazil, Switzerland]",Brazil,Switzerland,Cameroon,Switzerland
111,2022,Group H,0,initial,None,None,NaN,"[Ghana, Portugal]",Portugal,Ghana,South Korea,NaN
112,2022,Group H,1,26,Ghana,Uruguay,NaN,"[Uruguay, Portugal]",Portugal,Uruguay,Ghana,Uruguay


In [16]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\men\uefa\wc\standings_wc_uefa_men.xlsx'
final_composition_changes_df.to_excel(file_path, index=False)


# best four third placed

In [17]:
# Call the function to get the DataFrame
final_df = best_four_third_placed_wc_men(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day)

# Define the file path and save to Excel
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\men\uefa\wc\third_teams_wc_uefa_men.xlsx'
final_df.to_excel(file_path, index=False)



--- Processing Year: (1986,) ---

=== Initial Standings for Year (1986,) Before Processing Any Goals ===

            team   stage  before_last_game_points  before_last_game_goals_scored  before_last_game_goals_conceded  total_goal_difference
       Argentina Group A                      3.0                            4.0                              2.0                    2.0
        Bulgaria Group A                      2.0                            2.0                              2.0                    0.0
           Italy Group A                      2.0                            2.0                              2.0                    0.0
     South Korea Group A                      1.0                            2.0                              4.0                   -2.0
          Mexico Group B                      3.0                            3.0                              2.0                    1.0
        Paraguay Group B                      3.0                      

In [18]:
# Count the number of observations where the year is 1986
count_1986 = goals_last_day_sorted[goals_last_day_sorted['year'] == 1986].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 1986: {count_1986}")

# Count the number of observations where the year is 1990
count_1990 = goals_last_day_sorted[goals_last_day_sorted['year'] == 1990].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 1990: {count_1990}")

# Count the number of observations where the year is 1994
count_1994 = goals_last_day_sorted[goals_last_day_sorted['year'] == 1994].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 1994: {count_1994}")


Number of observations in goals_last_day_sorted for the year 1986: 38
Number of observations in goals_last_day_sorted for the year 1990: 11
Number of observations in goals_last_day_sorted for the year 1994: 116
